In [1]:
#!pip install -q git+https://github.com/deepmipt/bert.git@feat/keras tf-nightly-gpu deeppavlov

In [ ]:
import tensorflow as tf

In [3]:
tf.__version__

'1.14.1-dev20190524'

In [3]:
# tf.compat.v1.enable_eager_execution()

In [4]:
tf.compat.v1.executing_eagerly()

False

# Paraphrase data

In [5]:
BATCH_SIZE = 64  # both for training and evaluation for now
MAX_LEN = 52  # the longest sequence in the paraphrase dataset, workaroud that make shape of all batch tensors equal

In [7]:
tf.keras.utils.get_file(
    fname='paraphraser.zip',
    origin='http://files.deeppavlov.ai/datasets/paraphraser.zip',
    cache_subdir='dataset',
    extract=True,
    cache_dir='.'
)

'./dataset/paraphraser.zip'

In [8]:
tf.keras.utils.get_file(
    fname='paraphraser_gold.zip',
    origin='http://files.deeppavlov.ai/datasets/paraphraser_gold.zip',
    cache_subdir='dataset',
    extract=True,
    archive_format='zip',
    cache_dir='.'
)

'./dataset/paraphraser_gold.zip'

In [9]:
tf.keras.utils.get_file(
    fname='rubert_cased_L-12_H-768_A-12_v1.tar.gz',
    origin='http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz',
    cache_subdir='models',
    extract=True,
    cache_dir='.'
)

'./models/rubert_cased_L-12_H-768_A-12_v1.tar.gz'

In [6]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [7]:
def data_gen(train_data=True):
    from deeppavlov.dataset_readers.paraphraser_reader import ParaphraserReader
    from bert_dp.tokenization import FullTokenizer
    ds = ParaphraserReader().read(data_path='dataset', do_lower_case=False)
    tokenizer = FullTokenizer(vocab_file='models/rubert_cased_L-12_H-768_A-12_v1/vocab.txt', do_lower_case=False)
    tds = ds['train'] if train_data else ds['test']
    for pair, label in tds:
        t1 = tokenizer.tokenize(pair[0])
        t2 = tokenizer.tokenize(pair[1])
        _truncate_seq_pair(t1, t2, 512)
        s = ['[CLS]'] + t1 + ['[SEP]'] + t2 + ['[SEP]']
        yield tokenizer.convert_tokens_to_ids(s), label

In [8]:
train_ds = tf.data.Dataset.from_generator(data_gen,
                                          output_types=(tf.int32, tf.int32),
                                          output_shapes=(tf.TensorShape([None]), tf.TensorShape([])))


W0524 16:11:32.918092 139644132427584 deprecation.py:323] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py:494: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [9]:
batched_train_ds = train_ds.padded_batch(batch_size=BATCH_SIZE,
                                         padded_shapes=(MAX_LEN, # tf.constant([-1], dtype=tf.int64),
                                                        ()),
                                         drop_remainder=True)

In [10]:
test_ds = tf.data.Dataset.from_generator(data_gen,
                                         output_types=(tf.int32, tf.int32),
                                         output_shapes=(tf.TensorShape([None]), tf.TensorShape([])),
                                         args=(False,))

In [11]:
batched_test_ds = test_ds.padded_batch(batch_size=BATCH_SIZE,
                                       padded_shapes=(MAX_LEN,  # tf.constant([-1], dtype=tf.int64),
                                                      ()),
                                       drop_remainder=True)

# Model construction

In [12]:
from bert_dp import bert

In [13]:
# some data to pass through model in order to construct it
toy_inputs = tf.constant([[1, 3, 2, 5] * (MAX_LEN // 4),
                      [1, 2, 3] * (MAX_LEN // 4) + [0] *(MAX_LEN // 4)] * (BATCH_SIZE // 2))
bert_model = bert.BERT(name='bert')
bert_model(toy_inputs)

W0524 16:11:42.950564 139644132427584 deprecation_wrapper.py:119] From /home/nab/PycharmProjects/work/bert/bert_dp/embeddings.py:27: The name tf.keras.initializers.TruncatedNormal is deprecated. Please use tf.compat.v1.keras.initializers.TruncatedNormal instead.

W0524 16:11:42.951346 139644132427584 deprecation.py:506] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/keras/initializers.py:94: calling TruncatedNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


<tf.Tensor 'bert/pooler/dense/Tanh:0' shape=(64, 768) dtype=float32>

# Training

In [14]:
paraphrase_detector = tf.keras.Sequential([bert_model, tf.keras.layers.Dense(1, activation='sigmoid')])

In [15]:
from bert_dp.weight_decay_optimizers import AdamW

In [16]:
NUM_EPOCHS = 3

In [17]:
STEPS_PER_EPOCH = 112  # calculated by hand for now

In [18]:
learning_rate = tf.keras.optimizers.schedules.PolynomialDecay(initial_learning_rate=2e-05,
                                                              # n. of batches per epoch * ~NUM_EPOCHS for convergence 
                                                              decay_steps=STEPS_PER_EPOCH * NUM_EPOCHS,
                                                              end_learning_rate=1e-06)

In [19]:
paraphrase_detector.compile(optimizer=AdamW(weight_decay=0.01,
                                            learning_rate=learning_rate,
                                            epsilon=1e-6),  # as in bert_dp
                            loss='binary_crossentropy',
                            metrics=[tf.keras.metrics.BinaryAccuracy(),
                                     tf.keras.metrics.Precision(),
                                     tf.keras.metrics.Recall()])

In [20]:
paraphrase_detector.evaluate(batched_test_ds)

W0524 16:12:06.575785 139644132427584 deprecation.py:323] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0524 16:12:08.837022 139639889311488 deprecation_wrapper.py:119] From /home/nab/PycharmProjects/work/bert/bert_dp/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



     29/Unknown - 5s 176ms/step - loss: 0.7488 - binary_accuracy: 0.5981 - precision: 0.5981 - recall: 1.0000

[0.7487698330961424, 0.59806037, 0.59806037, 1.0]

In [21]:
bert_model.load_weights('models/rubert_converted_checkpoint/bert_model').assert_consumed()

In [22]:
paraphrase_detector.evaluate(batched_test_ds)

     29/Unknown - 5s 167ms/step - loss: 0.7340 - binary_accuracy: 0.4321 - precision: 0.6892 - recall: 0.0919

[0.7339627660554031, 0.43211207, 0.6891892, 0.09189189]

In [23]:
history = paraphrase_detector.fit(batched_train_ds.repeat(NUM_EPOCHS),
                                  epochs=NUM_EPOCHS,
                                  callbacks=[tf.keras.callbacks.TensorBoard(batch_size=BATCH_SIZE)],
#                                   validation_split=0.07,  # ~500 samples
                                  steps_per_epoch=STEPS_PER_EPOCH) # n. of batches in batched_train_ds

W0524 16:12:45.861352 139644132427584 deprecation.py:323] From /home/nab/PycharmProjects/work/venv/lib/python3.6/site-packages/tensorflow/python/keras/optimizer_v2/learning_rate_schedule.py:409: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Epoch 1/3
112/112 [==============================] - 49s 434ms/step - loss: 0.6939 - binary_accuracy: 0.6244 - precision: 0.6390 - recall: 0.9591
Epoch 2/3
112/112 [==============================] - 44s 392ms/step - loss: 0.6748 - binary_accuracy: 0.6445 - precision: 0.6445 - recall: 1.0000
Epoch 3/3
112/112 [==============================] - 44s 395ms/step - loss: 0.6916 - binary_accuracy: 0.6445 - precision: 0.6445 - recall: 1.0000


In [24]:
'f1 = ' + str((2 * history.history['precision'][-1] * history.history['recall'][-1]) / 
              (history.history['precision'][-1] + history.history['recall'][-1]))

'f1 = 0.7838479809976246'

In [26]:
eval_metrics = paraphrase_detector.evaluate(batched_test_ds)

     29/Unknown - 5s 180ms/step - loss: 0.6930 - binary_accuracy: 0.5981 - precision: 0.5981 - recall: 1.0000

In [27]:
'f1 = ' + str((2 * eval_metrics[2] * eval_metrics[3]) / 
              (eval_metrics[2] + eval_metrics[3]))

'f1 = 0.7484828244402932'